# Invoke-WebRequest
**Objective**: Scrape data from a webpage without an API service

**Website**: [Quotes to Scrape](quotes.toscrape.com)
- Quote
- Author
- Tags

<img src="https://github.com/MergeWords/PowerShell/blob/Invoke-WebRequest/Invoke-WebRequest/screencapture-quotes-toscrape-2020-06-12-02_16_10-edit.png?raw=true" width="90%">

## Step 1 - Inspect HTML structure

In [31]:
$WebSite = Invoke-WebRequest "http://quotes.toscrape.com/"
$WebSite



StatusCode        : 200
StatusDescription : OK
Content           : <!DOCTYPE html>
                    <html lang="en">
                    <head>
                    	<meta charset="UTF-8">
                    	<title>Quotes to Scrape</title>
                        <link rel="stylesheet" href="/static/bootstrap.min.css">
                        <link rel="stylesheet" href="/static/m...
RawContent        : HTTP/1.1 200 OK
                    Connection: keep-alive
                    X-Upstream: spidyquotes-master_web
                    Content-Length: 11053
                    Content-Type: text/html; charset=utf-8
                    Date: Fri, 12 Jun 2020 08:17:06 GMT
                    Server: nginx/1.14.0 (U...
Forms             : {}
Headers           : {[Connection, keep-alive], [X-Upstream, spidyquotes-master_web], [Content-Length, 11053], 
                    [Content-Type, text/html; charset=utf-8]...}
Images            : {}
InputFields       : {}
Links             : {@{i

Use the **Content** method to view all the HTML for the site

In [ ]:
$Website.Content

> - **Quote** is in [<_span_>](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/span) element
> - **Author** is in the [<_small_>](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/small) element
> - **Tags** are in the [<_meta_>](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/meta) element